In [153]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import time 
import random

# Classical Computing

In [208]:
# Define the length of the bit string that the algorithm willfind
s_length = 20
# Generate a bitstring of length 's_length'
s = random.getrandbits(s_length)

print('Bit string in decimal:', s)

Bit string in decimal: 397897


In [209]:
# Defines the classical function that is used in the algorithm.
# In this classical setting, we need to call this function 's_length'
# number of times since for every call, we can determine one correct position
# in the bit string 's'.
def classical_function(x):
	# Perform bitwise AND and sum the 1 bits.
	# returns 0 if the number of 1 bits in x & s is even and 1 
	# if the number of 1 bits is odd.
	return bin(x & s).count('1') % 2

In [210]:
def classical_version(length):
	s_prediction = 0 # Initilize it to 0

	for i in range(length):
		# Create a bitstring as an integer with a 1 at the i-th position (1 << i)
		bitstring_i = 1 << i
		# Query the black box with the integer bitstring and update the prediction.
		# Updates 's_prediction' by setting the i-th bit to the result of the query.
		s_prediction |= classical_function(bitstring_i) << i

	return format(s_prediction, f'0{length}b')  # Convert back to a binary string

# Quantum Computing

In [211]:
backend = Aer.get_backend('qasm_simulator')

In [212]:
# Quantum Oracle
def oracle(qc, s, num_bits):
	# Apply controlled-X (CNOT) gates for each bit of the hidden string s
	for i in range(num_bits):
		if (s >> i) & 1:  # If the i-th bit of s is 1
			qc.cx(i, num_bits)  # Apply a CNOT gate

In [213]:
def berstein_vazirani(s, num_bits):
	# Create a quantum circuit with (num_bits + 1) qubits and num_bits classical bits
	qc = QuantumCircuit(num_bits + 1, num_bits)

	# Initialize the ancilla qubit (last qubit) in the |1⟩ state, 
	# then apply a Hadamard gate to it
	# This puts it in the |-> state: (|0⟩ - |1⟩)/√2
	qc.x(num_bits)  # Apply X gate to flip the state from |0⟩ to |1⟩
	qc.h(range(num_bits + 1))  # Apply Hadamard gates to all qubits 

	# Apply the oracle function to encode the secret string s
	# The oracle flips the phase of the ancilla qubit depending 
	# on the bitwise dot product of the input qubits and s. Notice that
	# we only need to make one single call to te function.
	oracle(qc, s, num_bits)

	# Apply Hadamard gates to the first num_bits qubits
	# This operation effectively transforms the qubits back to 
	# the computational basis
	# and allows us to extract the secret string s
	qc.h(range(num_bits))

	# Measure the first num_bits qubits and store the result in 
	# the classical bits
	# The outcome will be the binary representation of the secret 
	# string s
	qc.measure(range(num_bits), range(num_bits))

	# Run the quantum circuit on a classical simulator (qasm_simulator)
	backend = Aer.get_backend('qasm_simulator')
	transpiled_qc = transpile(qc, backend)  # Optimize the circuit for the chosen backend
	job = backend.run(transpiled_qc, shots=1)  # Execute the circuit once 
	result = job.result().get_counts()  # Get the result in terms of counts 

	# Extract and return the result as a binary string
	# Since shots=1, there will be only one result in the dictionary, 
	# so we return the only key
	return next(iter(result))


In [214]:
print(f"Hidden bitstring: {format(s, f'0{s_length}b')}")

Hidden bitstring: 01100001001001001001


In [215]:
print(f"Found bitstring (classical method): {classical_version(s_length)}")

Found bitstring (classical method): 01100001001001001001


In [216]:
# Run the quantum algorithm
found_bitstring = berstein_vazirani(s, s_length)
print(f"Found bitstring (quantum method): {found_bitstring}")

Found bitstring (quantum method): 01100001001001001001
